# LinkedIn Data Cleaning Notebook

## Objective:
- Read files and convert xlsx into csv files
- Read LinkedIn CSV files and extract connections.
- Create a cleaned adjacency list (graph representation).
- Given Summary of graph using First Year Batch List- AllSU.csv
- Save the cleaned data as a JSON file.
- Log skipped rows for debugging.

---

## Manual Work
- Some students submitted their CSV files with file names that do not match their LinkedIn profile names so manual       cleaning of file names was needed to ensure accurate mapping with LinkedIn profiles.
- Extracted one zip file.

---

In [2]:
# Imports
import os
import csv
import json
from collections import defaultdict
import pandas as pd
import difflib
import shutil

- Reads .csv and .xlsx files from a folder.

- Converts .xlsx files into .csv.

- Matches each file with student names using fuzzy matching.

- Renames and copies them to a clean output folder.

In [35]:
# PATHS
input_folder = "LinkedIn Data Public"
student_csv = "AllSU.csv"
output_folder = "LinkedIn Data FILES"
os.makedirs(output_folder, exist_ok=True)

#LOAD STUDENT NAMES 
student_df = pd.read_csv(student_csv)
student_names = student_df['Full Name'].dropna().tolist()

# CLEANING FUNCTION 
def clean_string(s):
    s = s.replace("_", " ").strip().lower()
    return ''.join(e for e in s if e.isalnum() or e == ' ')

# TRACKERS
unmatched_files = []
skipped_files = []
converted_files = []

#RENAME FUNCTION
def rename_files():
    for file in os.listdir(input_folder):
        file_path = os.path.join(input_folder, file)

        # Skip hidden and invalid files
        if file.startswith('.') or not (file.endswith('.csv') or file.endswith('.xlsx')):
            skipped_files.append(file)
            continue

        #HANDLE .xlsx → .csv
        if file.endswith(".xlsx"):
            try:
                df = pd.read_excel(file_path)
                base_name = os.path.splitext(file)[0]
                csv_path = os.path.join(input_folder, f"{base_name}.csv")
                df.to_csv(csv_path, index=False)
                file_path = csv_path
                file = f"{base_name}.csv"
                converted_files.append(file)
            except Exception as e:
                print(f"Failed to convert {file} to CSV: {e}")
                skipped_files.append(file)
                continue

        base_name = os.path.splitext(file)[0]

        # Fuzzy matching with student names
        best_match = difflib.get_close_matches(
            clean_string(base_name),
            [clean_string(name) for name in student_names],
            n=1,
            cutoff=0.6
        )

        if best_match:
            matched_name = next(
                (name for name in student_names if clean_string(name) == best_match[0]), None
            )
            if matched_name:
                new_filename = f"{matched_name}.csv"
                new_path = os.path.join(output_folder, new_filename)
                shutil.copy2(file_path, new_path)
                print(f"{file} → {new_filename}")
            else:
                print(f"⚠ Match logic failed for {file}")
                unmatched_files.append(file)
        else:
            print(f"No close match for: {file}")
            unmatched_files.append(file)

    # FINAL REPORT
    if unmatched_files:
        print("\n The following files could not be matched to any student name:")
        for fname in unmatched_files:
            print(f" - {fname}")
    else:
        print("\n All files matched successfully!")

    if converted_files:
        print("\n Converted .xlsx files to CSV:")
        for fname in converted_files:
            print(f" - {fname}")

    if skipped_files:
        print("\nThe following files were skipped (not CSV/XLSX or hidden):")
        for fname in skipped_files:
            print(f" - {fname}")

# RUN
rename_files()


Swati_Kumari - Swati kumari.csv → Swati kumari.csv
Anand Kumar_Pandey - Anand Kumar Pandey.csv → Anand Kumar Pandey.csv
YuvrajSingh_Bhati - Yuvraj Bhati.csv → Yuvraj Singh Bhati.csv
Harisingh_Rajpoot - Harisingh Rajpoot.csv → Harisingh Rajpoot.csv
Divyanshi_Rathour - Divyanshi Rathour.csv → Divyanshi Rathour.csv
Aradhya_Patel - Aradhya Patel.csv → Aradhya Patel.csv
Abhishek_Singh - Abhishek Singh.csv → Abhishek Singh.csv
Mausam_kumari - Mausam kumari.csv → Mausam kumari.csv
Ashwin_Yadav - Ashwin Yadav.csv → Ashwin Yadav.csv
Anuradha_Tiwari - Anuradha Tiwari.csv → Anuradha Tiwari.csv
Shubham_Kang - Shubham Kang.csv → Shubham Kang.csv
Rahul_Kumar - Rahul Kumar.csv → Rahul Kumar.csv
Priyadarshi_Kumar - Priyadarshi Kumar.csv → Priyadarshi Kumar.csv
Shivang_Dubey - Shivang Dubey.csv → Shivang Dubey.csv
Sarthaksuman_Mishra - Sarthak Mishra.csv → SarthakSuman Mishra.csv
Lakhan_Rathore - Lakhan Rathore.csv → Lakhan Rathore.csv
Dilip_Suthar - DILIP SUTHAR.csv → Dilip Suthar.csv
ByagariPraveen_K

Create a cleaned adjacency list (graph representation).

In [36]:
#  Normalize Names
def normalize_name(name):
    """Convert name to lowercase and strip extra spaces to maintain consistency."""
    return " ".join(name.strip().lower().split())


In [37]:
def find_valid_headers(file_path):
    """Finds a valid header row with 'First Name' and 'Last Name', ignoring case and hidden characters."""
    with open(file_path, newline='', encoding='utf-8-sig', errors='replace') as csvfile:
        sample_reader = csv.reader(csvfile)
        for i, row in enumerate(sample_reader):
            # Normalize each column header
            cleaned = [col.strip().lower().replace('\u200b', '').replace('\xa0', '') for col in row]
            if "first name" in cleaned and "last name" in cleaned:
                return row  # Return original for DictReader
            if i >= 4:
                break
    return None


In [38]:
#  Create Adjacency List
def create_adjacency_list(folder_path):
    adjacency_list = defaultdict(set)
    skipped_rows_log = []
    csv_files = {normalize_name(os.path.splitext(f)[0]) for f in os.listdir(folder_path) if f.endswith(".csv")}

    for filename in os.listdir(folder_path):
        if not filename.endswith(".csv"):
            continue

        file_path = os.path.join(folder_path, filename)
        person_name = normalize_name(os.path.splitext(filename)[0])
        adjacency_list[person_name]

        headers = find_valid_headers(file_path)
        if not headers:
            print(f"⚠️ Warning: No valid headers found in '{filename}', skipping.")
            continue

        with open(file_path, newline='', encoding='utf-8-sig', errors='replace') as csvfile:
            reader = csv.DictReader(csvfile, fieldnames=headers)
            next(reader, None)

            for row_num, row in enumerate(reader, start=2):
                try:
                    first_name = normalize_name(row.get('First Name', ''))
                    last_name = normalize_name(row.get('Last Name', ''))
                    full_name = f"{first_name} {last_name}".strip()

                    if full_name:
                        adjacency_list[full_name]
                        adjacency_list[person_name].add(full_name)
                        adjacency_list[full_name].add(person_name)
                except Exception as e:
                    skipped_rows_log.append(f"Skipped row {row_num} in '{filename}': {e}")
                    continue

    adjacency_list = {k: sorted(list(v)) for k, v in adjacency_list.items()}
    return adjacency_list, csv_files, skipped_rows_log


In [39]:
# Save JSON Output
def save_json(data, file_path):
    """Save dictionary as a JSON file."""
    with open(file_path, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4)

In [40]:
# Extract (first, last) from name
def get_first_last(name):
    words = name.strip().lower().split()
    return (words[0], words[-1]) if words else ("", "")

# Run Cleaning Process
folder_path = "LinkedIn Data FILES"
adjacency_list, csv_files, skipped_rows_log = create_adjacency_list(folder_path)

if not adjacency_list:
    print("No connections found.")
else:
    output_file = "cleaned_adjacency_list.json"  # Save outside the folder
    save_json(adjacency_list, output_file)
    print(f"\n Cleaned adjacency list saved to {output_file}")
    print(f"Total nodes in cleaned graph: {len(adjacency_list)}")

    # Compare With Master List
    student_excel = "AllSU.csv"
    df_students = pd.read_csv(student_excel)

    # Normalize names and extract (first, last)
    df_students['Full Name'] = df_students['Full Name'].astype(str).apply(normalize_name)
    all_students_full = df_students['Full Name'].tolist()
    all_students_first_last = [get_first_last(name) for name in all_students_full]

    # Prepare keys
    json_nodes = {get_first_last(name) for name in adjacency_list.keys()}
    csv_files_first_last = {get_first_last(name) for name in csv_files}

    # Find missing sets
    missing_all = [name for name in all_students_first_last if name not in csv_files_first_last]
    missing_csv_but_in_json = sorted([name for name in missing_all if name in json_nodes])
    missing_csv_and_not_in_json = sorted([name for name in missing_all if name not in json_nodes])

    # Summary Report
    print(f"\nSummary:")
    print(f"Total CSV files found: {len(csv_files)}")
    print(f"Total students in AllSU list: {len(all_students_first_last)}")  # fixed this line
    print(f"Students who did NOT submit CSV: {len(missing_all)}")
    print(f"Students who did NOT submit CSV but appear as nodes: {len(missing_csv_but_in_json)}")
    for first, last in missing_csv_but_in_json:
        print(f"- {first} {last}")
    print(f"\n Students who did NOT submit CSV and also do NOT appear as nodes: {len(missing_csv_and_not_in_json)}")
    for first, last in missing_csv_and_not_in_json[:5]:
        print(f"- {first} {last}")

# Show Skipped Rows
if skipped_rows_log:
    print("\n--- Skipped Rows Log ---")
    for log in skipped_rows_log:
        print(log)



 Cleaned adjacency list saved to cleaned_adjacency_list.json
Total nodes in cleaned graph: 28555

Summary:
Total CSV files found: 126
Total students in AllSU list: 158
Students who did NOT submit CSV: 30
Students who did NOT submit CSV but appear as nodes: 28
- aslam khan
- avinash kumar
- deepak mandal
- dilip vaishnav
- gaurav bhargava
- gaurav kumar
- guman singh
- harsh chourasiya
- hirak nath
- jagriti pandey
- joel moirangthem
- mani kumar
- manish chhaba
- nikhil mehta
- nirmal kumar
- pawan kumar
- prem kushwaha
- rajeev yadav
- rakshita biradar
- ritesh jha
- ritesh yadav
- rohit kahar
- shalini priya
- shreyank sthavaramath
- surveer rao
- tamnna parveen
- vishal yadav
- yash yadav

 Students who did NOT submit CSV and also do NOT appear as nodes: 2
- abhilash rayala
- shivam sindhu
